In [1]:
# Imports
import math
from timeit import timeit
import torch as pt
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from matplotlib.widgets import Slider

print(pt.cuda.is_available())
pt.device("cuda" if pt.cuda.is_available() else "cpu")
pt.set_default_tensor_type('torch.cuda.FloatTensor')



True


c:\Users\alekh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [2]:
resolution = 2000

# Natural constants using atomic units
permittivity = 1    # Vacuum permittivity (4 pi epsilon _0), defined as one in atomic units
q = 1               # Elementary charge, defined as one in atomic units
electron_field_strength = q / permittivity # Becomes one in atomic units

# Other parameters
dim_size = 6   # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m


In [3]:
# Define wavefunctions

def psi(x1, x2, a1, a2, b):
    return pt.exp(- (x1 - a1)**2 / b) * pt.exp(- (x2 - a2)**2 / b) * (x1 - x2)

def psi_sq(x1, x2, a1, a2, b):
    return pt.exp(-2 * (x1 - a1)**2 / b) * pt.exp(-2 * (x2 - a2)**2 / b) * (x1 - x2) ** 2

def psi_sq_normalized(x1, x2, a1, a2, b):
    norm = pt.pi * b / 4 * (2 * (a1-a2)**2 + b)
    return psi_sq(x1, x2, a1, a2, b) / norm
    
def root1_psi_sq(x1, x2, a1, a2, b):
    return pt.exp(-2 * (x1 - a1)**2 / b) * pt.exp(-2 * (x2 - a2)**2 / b) * (pt.abs((x1-x2)))

def root2_psi_sq(x1, x2, a1, a2, b):
    return pt.exp(-2 * (x1 - a1)**2 / b) * pt.exp(-2 * (x2 - a2)**2 / b) * (pt.abs((x1-x2))**0.5)

def psi_sq_relative(r, R, a1, a2, b):
    return pt.exp(-2/(2*b)*(r+(a1-a2))**2) * r**2 * pt.exp(-4/b * (R - 1/2 * (a1 + a2))**2)

def psi_sq_relative_normalized(r, R, a1, a2, b):
    norm = 1/2 * np.sqrt(b * pt.pi) * (2 * (a1-a2)**2 + b) * np.sqrt(pt.pi * b) / 2
    return psi_sq_relative(r, R, a1, a2, b) / norm

def phi_sq(p1, p2, a1, a2, b):
    return ((a2-a1)**2/b + (p1-p2)**2 / 4) * pt.exp(-b * p1**2 / 2 - a1**2/(2*b) -b * p2**2 / 2 - a2**2/(2*b))

def phi_sq_normalized(p1, p2, a1, a2, b):
    norm = (1+2 * (a1-a2)**2) * pt.pi / (b**2)
    return phi_sq(p1, p2, a1, a2, b) / norm

def psi_d2_dx12(x1, x2, a1, a2, b):
    exp = pt.exp(-(x1-a1)**2/b) * pt.exp(-(x2-a2)**2/b) 
    # f = 4*a1 - 6*x1 + 2*x2 - (4*(x2-x1)*(x1-a1)**2) / b**2 - (4*a2 - 6*x2 + 2*x1 - (4*(x1-x2)*(x2-a2)**2) / b**2)  
    f1 =  1/b * (-6*x1  + 4*x1**3/b - 8*a1*x1**2/b + 4*a1**2*x1/b + 4*a1 + 2*x2 - 4*x1**2*x2/b + 8*x1*x2*a1/b - 4*x2*a1**2/b)
    f2 = -1/b * (-6*x2  + 4*x2**3/b - 8*a2*x2**2/b + 4*a2**2*x2/b + 4*a2 + 2*x1 - 4*x2**2*x1/b + 8*x2*x1*a2/b - 4*x1*a2**2/b)
    
    f1 =  (4 * x1**3 + (-4 * x2 - 8 * a1) * x1**2 + (8 * a1 * x2 - 6 * b + 4 * a1**2) * x1 + (2 * b - 4 * a1**2) * x2 + 4 * a1 * b) / b**2
    f2 = -(4 * x2**3 + (-4 * x1 - 8 * a2) * x2**2 + (8 * a2 * x1 - 6 * b + 4 * a2**2) * x2 + (2 * b - 4 * a2**2) * x1 + 4 * a2 * b) / b**2
    return exp * (f1 + f2)


In [4]:
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-2, 2, 300)
x2 = pt.linspace(-2, 2, 300)
X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
# pos = ax.imshow(va.get_pot(X1, X2, kappa, theta_ext, E_ext, d, h).cpu(), extent=[-dim_size, dim_size, -dim_size, dim_size])
# cbar = fig.colorbar(pos, ax=ax)

Z = pt.load("results_physical.pt")
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')

ax.set_xlabel(r'x1 [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_ylabel(r'x2 [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'E [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()  

C:\Users\alekh\AppData\Local\Temp\ipykernel_7620\2662795652.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_physical.pt")


In [6]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 300)
x2 = pt.linspace(-3, 3, 300)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_unphysical.pt")
# Print location Z minima
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

C:\Users\alekh\AppData\Local\Temp\ipykernel_24108\3488573912.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_unphysical.pt")


In [7]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 400)
x2 = pt.linspace(-3, 3, 400)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_intermediate08.pt")
# Print location Z minima
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

C:\Users\alekh\AppData\Local\Temp\ipykernel_24108\4110490742.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_intermediate08.pt")


In [8]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 400)
x2 = pt.linspace(-3, 3, 400)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_intermediate1.pt")
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

C:\Users\alekh\AppData\Local\Temp\ipykernel_24108\2162636906.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_intermediate1.pt")


In [5]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 400)
x2 = pt.linspace(-3, 3, 400)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_intermediate15.pt")
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

C:\Users\alekh\AppData\Local\Temp\ipykernel_7620\2957908699.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_intermediate15.pt")


fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 400)
x2 = pt.linspace(-3, 3, 400)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_intermediate15.pt")
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

In [10]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 400)
x2 = pt.linspace(-3, 3, 400)
# h = 1
X1, X2 = pt.meshgrid(x1, x2, indexing='ij')
Z = pt.load("results_intermediate2.pt")
ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()*1000, cmap='viridis')
plt.xlabel(r'$\mu_1$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
plt.ylabel(r'$\mu_2$ [$\text{a}_\text{0}$]') # In units of bohr radius = 4 pi epsilon_0 h_bar^2 / (m_e e+^2), = 5.2917e-11 m
ax.set_zlabel(r'$\text{E}$ [$\text{E}_\text{h} x10^{-3}$]') # In units of hartree energy, with E_h = e^2 / (4 pi epsilon_0 a_0) = 4.3597e-18 J
plt.show()

C:\Users\alekh\AppData\Local\Temp\ipykernel_17468\811877557.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Z = pt.load("results_intermediate2.pt")


In [16]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 300)
x2 = pt.linspace(-3, 3, 300)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')

mu1 = 0
mu2 = 0.5
b = 2

Z = psi_sq_normalized(X1, X2, mu1, mu2, b)

ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu(), cmap='viridis')
# plot vertical line
x1 = b * (2 * mu1 - mu2 +  2 * mu1 * (mu1-mu2) ** 2) / (2 * (mu1 - mu2) ** 2 + b)
x2 = b * (2 * mu2 - mu1 +  2 * mu2 * (mu2-mu1) ** 2) / (2 * (mu2 - mu1) ** 2 + b)
ax.plot([x1, x1], [x2, x2], [0, 0.5], color='red')
plt.show()


In [10]:
fig = plt.figure()
ax = plt.axes(projection = "3d")
x1 = pt.linspace(-3, 3, 300)
x2 = pt.linspace(-3, 3, 300)

X1, X2 = pt.meshgrid(x1, x2, indexing='ij')

mu1 = 0
mu2 = 0.5
b = 2

Z = root1_psi_sq(X1, X2, mu1, mu2, b)
Z2 = root2_psi_sq(X1, X2, mu1, mu2, b)
Z3 = psi_sq_normalized(X1, X2, mu1, mu2, b)

ax.plot_surface(X1.cpu(), X2.cpu(), Z.cpu()/pt.sum(Z.cpu()), cmap='viridis')
# ax.plot_surface(X1.cpu(), X2.cpu(), Z2.cpu()/pt.sum(Z2.cpu()), cmap='Greens')
ax.plot_surface(X1.cpu(), X2.cpu(), Z3.cpu()/pt.sum(Z3.cpu()), cmap='coolwarm')

plt.show()
